### 作業目的: 使用N-Gram模型預測文字

本次作業會使用[桃園市官網市政新聞](https://data.gov.tw/dataset/25891)來進行練習

### 載入所需的Libraries

In [1]:
import json
import re
from collections import Counter, namedtuple

### 載入資料

In [3]:
with open('./WebNews.json', 'r', encoding="utf-8") as f:
    news_data = json.load(f)

In [4]:
news_data[:2]

[{'file': [],
  'link': [],
  'id': '202006110003',
  'istop': 'N',
  'img': [{'imgname': '0611桃園屏東_200611_0004.jpg',
    'imgcontent': '<br>鄭<br>市<br>長<br>與<br>潘<br>縣<br>長<br>合<br>體<br>推<br>廣<br>防<br>疫<br>互<br>惠<br>旅<br>遊<br>',
    'imgurl': 'http://www.tycg.gov.tw/uploaddowndoc?file=news/202006111817400.jpg&filedisplay=202006111817400.jpg&flag=pic'},
   {'imgname': '0611桃園屏東_200611_0005.jpg',
    'imgcontent': '<br>潘<br>縣<br>長<br>挑<br>選<br>屏<br>東<br>特<br>色<br>伴<br>手<br>禮<br>致<br>贈<br>鄭<br>市<br>長<br>',
    'imgurl': 'http://www.tycg.gov.tw/uploaddowndoc?file=news/202006111817401.jpg&filedisplay=202006111817401.jpg&flag=pic'},
   {'imgname': '0611桃園屏東_200611_0007.jpg',
    'imgcontent': '<br>鄭<br>市<br>長<br>精<br>心<br>準<br>備<br>桃<br>園<br>特<br>色<br>伴<br>手<br>禮<br>送<br>給<br>潘<br>縣<br>長<br>',
    'imgurl': 'http://www.tycg.gov.tw/uploaddowndoc?file=news/202006111817402.jpg&filedisplay=202006111817402.jpg&flag=pic'},
   {'imgname': '0611桃園屏東_200611_0006.jpg',
    'imgcontent': '<br>桃<br>園<br

In [8]:
news_data[0]['detailcontent']

'<strong>迎接國旅爆發期，五星縣市長合體推廣桃園、屏東互惠旅遊</strong><br />&nbsp;<br />桃園市長鄭文燦今（11）日下午出席「桃園加屏東『旅遊優惠強棒一棒接一棒』好康發表記者會」，鄭市長表示，隨著疫情趨緩，國內進入國旅爆發期，觀光局近期公布最新10大熱門旅遊景點，桃園包辦前三名，依序為大溪老街、石門水庫及角板山，顯示桃園旅遊的潛力無限；鄭市長說，桃園市與屏東縣各自擁有獨特的觀光魅力與資源，希望能搭配中央觀光振興方案，共同推廣防疫互惠旅遊，並推出更多加碼優惠措施，降低民眾的旅遊負擔、增加旅遊的樂趣，也讓國境之南與國境之門做最好的交流。<br />&nbsp;<br />鄭市長提到，桃園是國門之都、屏東是國境之南，兩個城市各有特色及魅力，許多景點也互相輝映，假如你喜歡屏東的山川琉璃吊橋，絕對不能錯過桃園的新溪口吊橋，如果你喜歡大溪老街，也應該去恆春古城漫遊；鄭市長說，許多民眾造訪過很多國外景點，例如普吉島、巴里島，卻沒有到過小琉球、澎湖等離島，十分可惜；他也稱讚屏東幅員遼闊，擁有多樣化的景點，墾丁更是國內旅遊勝地之一，希望現階段推動防疫新生活的同時，桃園也能與各縣市合作，一起推廣防疫互惠旅遊，讓更多在地旅遊業者受惠。<br />&nbsp;<br />桃園預估下半年包含鄰里長、環保志工、守望相助隊、義警民防等觀摩旅遊團數，出團數將達500團以上，目前桃園第一階段已經與基隆、台南、屏東等縣市簽署合作契約，第二階段將與宜蘭、花蓮、南投、嘉義等縣市合作，希望未來逐漸擴大至其他縣市，提供遊客更多優惠措施；鄭市長也說，屏東縣推出三鐵優惠方案，桃園市自7月1日起推出「來桃園住一晚‧加贈免費一日遊」方案，遊客來桃園在合法旅宿住一晚，即可獲得市府加贈免費一日遊行程，內容包含遊覽車接送、午餐、景點門票或DIY體驗等，期盼帶領遊客體驗桃園的觀光魅力；此外，民眾也可以持200元振興三倍券兌換400元夜市券，藉此鼓勵民眾到夜市消費，市府未來也將陸續推出更多的加碼優惠措施。<br />&nbsp;<br />屏東縣長潘孟安則希望在鄭市長推廣下，吸引更多桃園市民到屏東旅遊，他表示，桃園為國境之門，擁有很大的發展優勢，在鄭市長帶領下，桃園連續多年結婚率及出生率都是全國之冠，也成為一座宜居城市，他也特別推薦親子旅遊首選屏東，屏東縣政府整合轄內旅宿業者、伴手禮業者、食品

In [28]:
test = [d['detailcontent'] for d in news_data][0]
test 

'<strong>迎接國旅爆發期，五星縣市長合體推廣桃園、屏東互惠旅遊</strong><br />&nbsp;<br />桃園市長鄭文燦今（11）日下午出席「桃園加屏東『旅遊優惠強棒一棒接一棒』好康發表記者會」，鄭市長表示，隨著疫情趨緩，國內進入國旅爆發期，觀光局近期公布最新10大熱門旅遊景點，桃園包辦前三名，依序為大溪老街、石門水庫及角板山，顯示桃園旅遊的潛力無限；鄭市長說，桃園市與屏東縣各自擁有獨特的觀光魅力與資源，希望能搭配中央觀光振興方案，共同推廣防疫互惠旅遊，並推出更多加碼優惠措施，降低民眾的旅遊負擔、增加旅遊的樂趣，也讓國境之南與國境之門做最好的交流。<br />&nbsp;<br />鄭市長提到，桃園是國門之都、屏東是國境之南，兩個城市各有特色及魅力，許多景點也互相輝映，假如你喜歡屏東的山川琉璃吊橋，絕對不能錯過桃園的新溪口吊橋，如果你喜歡大溪老街，也應該去恆春古城漫遊；鄭市長說，許多民眾造訪過很多國外景點，例如普吉島、巴里島，卻沒有到過小琉球、澎湖等離島，十分可惜；他也稱讚屏東幅員遼闊，擁有多樣化的景點，墾丁更是國內旅遊勝地之一，希望現階段推動防疫新生活的同時，桃園也能與各縣市合作，一起推廣防疫互惠旅遊，讓更多在地旅遊業者受惠。<br />&nbsp;<br />桃園預估下半年包含鄰里長、環保志工、守望相助隊、義警民防等觀摩旅遊團數，出團數將達500團以上，目前桃園第一階段已經與基隆、台南、屏東等縣市簽署合作契約，第二階段將與宜蘭、花蓮、南投、嘉義等縣市合作，希望未來逐漸擴大至其他縣市，提供遊客更多優惠措施；鄭市長也說，屏東縣推出三鐵優惠方案，桃園市自7月1日起推出「來桃園住一晚‧加贈免費一日遊」方案，遊客來桃園在合法旅宿住一晚，即可獲得市府加贈免費一日遊行程，內容包含遊覽車接送、午餐、景點門票或DIY體驗等，期盼帶領遊客體驗桃園的觀光魅力；此外，民眾也可以持200元振興三倍券兌換400元夜市券，藉此鼓勵民眾到夜市消費，市府未來也將陸續推出更多的加碼優惠措施。<br />&nbsp;<br />屏東縣長潘孟安則希望在鄭市長推廣下，吸引更多桃園市民到屏東旅遊，他表示，桃園為國境之門，擁有很大的發展優勢，在鄭市長帶領下，桃園連續多年結婚率及出生率都是全國之冠，也成為一座宜居城市，他也特別推薦親子旅遊首選屏東，屏東縣政府整合轄內旅宿業者、伴手禮業者、食品

In [31]:
test2 = re.findall(r'^<.\w*>$|[\u4E00-\u9FA50-9]', test)
test2[-10:]

['冷', '紅', '等', '均', '一', '同', '出', '席', '活', '動']

### 進行資料清洗
觀察上面的資料，資料包含許多其他的資訊，我們需要將真正的新聞內文取出，並且對內文進行文字清洗。
請做以下的文字處理:

1. 取出新聞內文
2. 去除HTML Tags
3. 移除標點符號，只保留英文、數字、中文

In [41]:
#取出新聞內文
corpus_list = [content['detailcontent'] for content in news_data]

#去除HTML Tags與標點符號(只保留英文、數字、中文)
corpus_list = [''.join(re.findall(r'^<.*?>$|[\u4E00-\u9FA50-9]', corpus)) for corpus in corpus_list]
corpus_list[2]

'配合中央準公共政策持續建構平價且完善的幼教服務桃園市長鄭文燦今11日上午前往龍潭區雙龍國小出席109學年度準公共幼兒園說明會時表示桃園人口平均年齡約38歲是非常年輕的城市且出生率居六都最高每年以約萬人口速度持續成長成為年輕家庭圓夢的首選城市面對幼教需求提高市府為照顧更多年輕家庭率先全國提出育兒津貼及托育公共化等創新政策成效良好並獲中央採納後續中央也逐步統一全國福利政策提出準公共政策盼建構平價且完善的幼教環境市府將持續加強改善不足之處及建立良好的公私夥伴關係盼中央給予更多支持讓年輕家庭甜蜜多一點負擔少一點讓幼教產業發展更好共同打造桃園成為準公共的領航城市鄭市長指出桃園目前人口數已超越225萬人成長快速且幼教需求提高因此增加公立及非營利幼兒園數量外亦推動私立幼兒園普及化並配合中央準公共政策上路鼓勵公私合作提供更完善的幼教服務桃園目前約有188家私幼符合準公共申請條件截至目前共94間完成簽約加入準公共的行列簽約率已達5成為全國第一私托中心簽約率高達約95居家托育人員簽約率則近9成提供家長更多新的托育選擇外同時大幅改善幼托環境讓年輕人有感進而提升桃園社福認同感鄭市長提及109年準公共教保服務機構合作要件規定園長中心主任教師及教保員薪資每月固定薪資至少2萬9000元以上幼兒園並應訂定調薪機制兼顧平價幼教及保障薪資兩原則下平衡處理市府亦將落實建物公安檢查協助業者以達標為目標提升綜合防災能力及保障托育環境安全讓幼教產業獲得更合理的支持鄭市長也談到109年準公共政策額外提供修繕或汰換遊戲場設施設備補助及增加招收2歲幼兒獎勵金目前私立幼兒園可向下延伸招收2歲專班另私托中心可收2至3歲幼兒市府鼓勵業者加入準公共政策於去108年編列約2320萬簽約獎勵金並常態性鼓勵幼兒園設備更新協助業者提升包括廚房廁所遊戲場等設備完善托育環境另外市府也實施腸病毒停班補助避免隱匿疫情及減少幼教業者損失盼與民間業者建立良好夥伴關係協助及陪伴幼教產業發展更好教育部國民及學前教育署長彭富源表示行政院在107年7月起推動少子女化對策計畫辦理準公共教保服務政策以有兩名新生兒以下的家庭為例參加每月收費1萬元以下的準公共教保服務機構政府將補助每月最高5500元家長每月繳費則不超過4500元期盼透過此類政策降低家長們的負擔彭署長也說桃園首創提出多項創新幼教福利政策方向明確且成效良好中央也因此跟進採納且統一全國推動準公共

### 建立N-Gram模型
N-Gram模型在計算詞機率時為(以Trigram為例)
$$
P(W_i|W_{i-1},W_{i-2}) = \frac{count(W_i,W_{i-1},W_{i-2})}{count(W_{i-1},W_{i-2})}
$$

舉例來說
$$
P(the|this,is) = \frac{count(this\ is\ the)}{count(this\ is)}
$$

In [42]:
def ngram(documents, N=2):
    
    #建立儲存預測字, 所有ngram詞頻字典, 所有字詞(分母)
    ngram_prediction = dict()
    total_grams = list()
    words = list()
    Word = namedtuple('Word', ['word', 'prob']) #使用namedtuple來儲存預測字詞與對應機率

    for doc in documents:
        # 在每個文章錢加上起始(<start>)與結束符號(<end>)
        split_words = ['<s>'] + list(doc) + ['</s>']
        # 計算分子
        [total_grams.append(tuple(split_words[i:i+N])) for i in range(len(split_words)-N+1)]
        # 計算分母
        [words.append(tuple(split_words[i:i+N-1])) for i in range(len(split_words)-N+2)]
    
    #計算分子詞頻
    total_word_counter = Counter(total_grams)
    #計算分母詞頻
    word_counter = Counter(words)
    
    #計算所有N-gram預測字詞的機率
    for key in total_word_counter:
        word = ''.join(key[:N-1])
        if word not in ngram_prediction:
            ngram_prediction.update({word: set()})
            
        next_word_prob = total_word_counter[key]/word_counter[key[:N-1]]
        w = Word(key[-1], f'{next_word_prob}')
        ngram_prediction[word].add(w)
        
    return ngram_prediction

### 使用N-Gram模型進行預測
這裡我們使用4 gram模型，也就是輸入三個字之後，第四個字出現的機率，並將輸出依據機率做排序

In [48]:
#建立four_gram模型，並將預測的機率按照大小排列
four_gram_pred = ngram(corpus_list, N=3)
for word, pred in four_gram_pred.items():
    four_gram_pred[word] = sorted(pred, key=lambda x: x.prob, reverse=True)

In [49]:
# 執行時因為字典數龐大，顯示會較慢
four_gram_pred

{'<s>迎': [Word(word='接', prob='1.0')],
 '迎接': [Word(word='2', prob='0.12'),
  Word(word='新', prob='0.12'),
  Word(word='疫', prob='0.12'),
  Word(word='開', prob='0.12'),
  Word(word='鼠', prob='0.08'),
  Word(word='未', prob='0.04'),
  Word(word='更', prob='0.04'),
  Word(word='天', prob='0.04'),
  Word(word='城', prob='0.04'),
  Word(word='1', prob='0.04'),
  Word(word='聖', prob='0.04'),
  Word(word='國', prob='0.04'),
  Word(word='4', prob='0.04'),
  Word(word='長', prob='0.04'),
  Word(word='走', prob='0.04'),
  Word(word='桃', prob='0.04')],
 '接國': [Word(word='道', prob='0.5'), Word(word='旅', prob='0.5')],
 '國旅': [Word(word='遊', prob='0.4'),
  Word(word='爆', prob='0.13333333333333333'),
  Word(word='以', prob='0.06666666666666667'),
  Word(word='8', prob='0.06666666666666667'),
  Word(word='市', prob='0.06666666666666667'),
  Word(word='商', prob='0.06666666666666667'),
  Word(word='方', prob='0.06666666666666667'),
  Word(word='行', prob='0.06666666666666667'),
  Word(word='卡', prob='0.0666666666

In [52]:
#給定字詞，使用ngram預測下一個字的機率(顯示top 10)
text = '市長'
next_words = list(four_gram_pred[text])[:10]
for next_word in next_words:
    print('next word: {}, probability: {}'.format(next_word.word, next_word.prob))

next word: 表, probability: 0.20354166666666668
next word: 鄭, probability: 0.19166666666666668
next word: 也, probability: 0.16666666666666666
next word: 指, probability: 0.095
next word: 說, probability: 0.09458333333333334
next word: 提, probability: 0.055625
next word: 回, probability: 0.023541666666666666
next word: 談, probability: 0.01125
next word: 李, probability: 0.010625
next word: 在, probability: 0.010416666666666666


可自行嘗試使用不同的N搭建不同的N-Gram模型的效果